In [6]:
import numpy as np
## test numpy matrix multiplication with Boolean values
identity_matrix = np.identity(4, dtype=bool)
identity_matrix[2][1] = True
identity_matrix[1][2] = True

test_x = np.ones((4, 1), dtype=bool)
test_x[1] = False
test_x[2] = False

answer = np.dot(identity_matrix, test_x)
print(answer)
print(np.sum(answer))


[[ True]
 [False]
 [False]
 [ True]]
2


In [123]:
# network size N
N = 3

# number of edges
M = N*3

## initialize time
T = 0
## choose Tmax
Tmax = 5


In [131]:
## inputs are the initial condition and the edge matrix
def find_extinction_time(b, E, x0):

    # ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    A = np.identity(N, dtype=bool)

    ## initialize number infected (based on initial conditions))
    I = np.sum(x0)

    ## initialize list of infected people for all timesteps
    infected = np.zeros(n)

    # run simulation
    for t in range(n):

        Iden = np.identity(N, dtype=bool)
        
        ## recovery event occurs (turns diagonal entry to 0 and premultiplies A) == row i of A is set to False
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)
            A[i, :] = False

            print(f"recovery event at time {t} of node {i}")

        else:
        ## spreading event (turns symmetric entries to 1 and premultiplies A)

            i,j = E[np.random.randint(M)]
            print(f"spreading event at time {t} of nodes {i} and {j}")

            Iden[i][j] = Iden[j][i] = True

            A = np.dot(Iden, A)
        
        infected[t] = np.sum(np.dot(A, x0))

        ## if zero infected people, break simulation, truncate infected array
        if np.all(A == 0) == True:

            infected = infected[:t]
            
            break

    return A, infected


The code above returned the number of infected individuals, but our algorithm doesn't need to keep track of this at the moment. Let us just return A now that we have convinced ourselves this is working as we expected. The most expensive operation is the np.dot(Iden, A) for each spreading event or recovery event; ideal to replace with a more efficient operation. If the premultiplying matrix has a spreading process between nodes i and j, we can replace rows i and j of the matrix A with the results of an OR operation on rows i and j of the matrix, leaving all other rows the same. Similarly, for a recoveory process, if the premultiplying matrix has the recovery of node i, replace the row i of matrix A with a row of zeros. 

In [152]:
## inputs are the initial condition and the edge matrix
def find_tranformation_matrix(b, E, x0):

    # ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    A = np.identity(N, dtype=bool)

    # run simulation
    for t in range(n):

        Iden = np.identity(N, dtype=bool)
        
        ## recovery event occurs (turns diagonal entry to 0 and premultiplies A) == row i of A is set to False
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)
            A[i, :] = False

            print(f"recovery event at time {t} of node {i}")

        else:
        ## spreading event (turns symmetric entries to 1 and premultiplies A) == row i and j of A is set to the OR of row i and j

            i,j = E[np.random.randint(M)]
            print(f"spreading event at time {t} of nodes {i} and {j}")

            A[i] = A[j] = np.logical_or(A[i], A[j])

        ## if zero infected people, break simulation, truncate infected array
        if np.all(A == 0) == True:
            
            break

    return A


In [158]:
b = 0.5

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

x0 = np.ones(N, dtype=bool)

matrix_A = find_tranformation_matrix(b, E, x0)

print(matrix_A)


spreading event at time 0 of nodes 2 and 0
spreading event at time 1 of nodes 2 and 0
spreading event at time 2 of nodes 1 and 0
recovery event at time 3 of node 1
recovery event at time 4 of node 1
recovery event at time 5 of node 0
spreading event at time 6 of nodes 2 and 1
spreading event at time 7 of nodes 1 and 2
recovery event at time 8 of node 1
spreading event at time 9 of nodes 1 and 0
spreading event at time 10 of nodes 2 and 1
recovery event at time 11 of node 2
spreading event at time 12 of nodes 2 and 0
spreading event at time 13 of nodes 1 and 0
recovery event at time 14 of node 2
spreading event at time 15 of nodes 2 and 1
recovery event at time 16 of node 0
spreading event at time 17 of nodes 0 and 0
recovery event at time 18 of node 1
spreading event at time 19 of nodes 1 and 1
spreading event at time 20 of nodes 0 and 0
recovery event at time 21 of node 2
[[False False False]
 [False False False]
 [False False False]]


Test:

In [76]:

A = np.identity(3, dtype=bool)


## first, let's choose a recovery of node 2 at time 0
Iden = np.identity(3, dtype=bool)
Iden[2][2] = False
A = np.dot(Iden, A)

## next a spreading event from node 0 to node 2 at time 1
Iden = np.identity(3, dtype=bool)
Iden[0][2] = Iden[2][0] = True
A = np.dot(Iden, A)


## recover 1 at time 2
Iden = np.identity(3, dtype=bool)  
A = np.dot(Iden, A)

# recover node 0 at time 3
Iden = np.identity(3, dtype=bool)
Iden[0][0] = False
A = np.dot(Iden, A)

## spread from node 0 to 1 at time 4
Iden = np.identity(3, dtype=bool)
Iden[0][1] = Iden[1][0] = True
A = np.dot(Iden, A)

print(A)


[[False False False]
 [False False False]
 [ True False False]]
